### Imports + Setup

In [57]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re  # Import the regular expression module
from tqdm import tqdm
import numpy as np


output_path = "../output/"

### 4.1   Clean df demographics

In [17]:
df_demographics = pd.read_csv(output_path + 'demographics_data.csv')

#### a)

In [ ]:
# ((df_demographics.LifeExpectancy_Both.isna()) |
#  (40 > df_demographics.LifeExpectancy_Both) |
#  (df_demographics.LifeExpectancy_Both > 100) |
#  df_demographics.LifeExpectancy_Both.isnumeric())

# Convert all relevant columns to numeric (except Country)
numeric_cols = df_demographics.columns.drop("Country")
df_demographics[numeric_cols] = df_demographics[numeric_cols].apply(pd.to_numeric, errors='coerce')


               Country
0          Afghanistan
1              Albania
2              Algeria
3               Angola
4  Antigua and Barbuda
object


In [6]:
dir(df_demographics.LifeExpectancy_Both)

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__column_consortium_standard__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pandas_priority__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__

b)

In [ ]:
# Filter conditions for valid life expectancy
mask_valid_life = (
    df_demographics["LifeExpectancy_Both"].notna() &
    (df_demographics["LifeExpectancy_Both"] >= 40) &
    (df_demographics["LifeExpectancy_Both"] <= 100)
)

# Apply filter
df_demographics = df_demographics[mask_valid_life].copy()


               Country
0          Afghanistan
1              Albania
2              Algeria
3               Angola
4  Antigua and Barbuda
object


C)

In [23]:
# Ensure proper numeric conversion and handle "None" or invalids
urban_cols = ["UrbanPopulation_Percentage", "UrbanPopulation_Absolute", "PopulationDensity"]
df_demographics[urban_cols] = df_demographics[urban_cols].apply(pd.to_numeric, errors='coerce')



d)

In [ ]:
# Make a copy of original names for mismatch logging
df_demographics["OriginalCountry"] = df_demographics["Country"]

# Normalize names
df_demographics["Country"] = (
    df_demographics["Country"]
    .str.strip()
    .str.replace(r"^the\s+", "", case=False, regex=True)  # remove "the "
    .str.title()
)


               Country
0          Afghanistan
1              Albania
2              Algeria
3               Angola
4  Antigua And Barbuda
object


e)

In [26]:
# Create a DataFrame of name mismatches
name_mismatches = df_demographics[df_demographics["OriginalCountry"] != df_demographics["Country"]][
    ["OriginalCountry", "Country"]
]

# Save mismatches to file
name_mismatches.to_csv(output_path + "name_mismatches.csv", index=False)


f)

In [34]:
# Drop OriginalCountry column and set Country as index
if "OriginalCountry" in df_demographics.columns:
    df_demographics.drop(columns="OriginalCountry", inplace=True)
if df_demographics.index.name != "Country":
    df_demographics.set_index("Country", inplace=True)


## 4.2) Clean df gdp

a)

In [42]:
df_gdp = pd.read_csv('../gdp_per_capita_2021.csv')
# Remove commas or special characters and convert to numeric
df_gdp['GDP_per_capita_PPP'] = (
    df_gdp['GDP_per_capita_PPP']
    
    # Convert all values to string type to allow regex operations
    .astype(str)
    
    # Use regular expression to remove all characters that are not digits or decimal points
    .str.replace(r'[^\d\.]', '', regex=True)
    
    # Replace any resulting empty strings (e.g., originally "None") with proper missing value (NA)
    .replace('', pd.NA)
)

# Convert the cleaned strings to actual numeric (float) values
# Any remaining invalid formats are coerced to NaN
df_gdp['GDP_per_capita_PPP'] = pd.to_numeric(df_gdp['GDP_per_capita_PPP'], errors='coerce')


b)

In [44]:
# Document rows with missing GDP
dropped_gdp = df_gdp[df_gdp['GDP_per_capita_PPP'].isna()]
dropped_gdp.to_csv(output_path + "dropped_gdp.csv", index=False)

# Drop them from the dataset
df_gdp.dropna(subset=["GDP_per_capita_PPP"], inplace=True)


c)

In [58]:
Q1 = df_gdp['GDP_per_capita_PPP'].quantile(0.25)
Q3 = df_gdp['GDP_per_capita_PPP'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df_gdp[
    (df_gdp['GDP_per_capita_PPP'] < lower_bound) |
    (df_gdp['GDP_per_capita_PPP'] > upper_bound)
]

print(f"Number of outliers in GDP data: {len(outliers)}")
print (f"Outliers:\n{outliers}")


Number of outliers in GDP data: 6
Outliers:
            GDP_per_capita_PPP
Country                       
Bermuda              92276.234
Ireland             118102.766
Luxembourg          137947.340
Norway               89209.140
Qatar               116832.570
Singapore           131864.100


d)

In [ ]:
# Duplicate Handling Documentation
# Check for countries that appear more than once.
# If duplicates are found, we:
#   - Save all duplicate entries to 'duplicate_gdp_entries.csv' for record keeping.
#   - Retain only the first occurrence of each country using `keep='first'`.
#     This is based on the assumption that the first entry is correct or representative,
#     and no additional criteria (e.g., recency, source reliability) are available to decide otherwise.
# This ensures consistency in merging/joining with other datasets later.

# Check for duplicates
duplicates = df_gdp[df_gdp.duplicated(subset=['Country'], keep=False)]

if not duplicates.empty:
    print("Duplicate entries found:\n", duplicates)
    print("The Population dataset contains duplicate country entries.")
    print("Decision: Keeping only the first occurrence of each duplicated country.")
    df_pop = df_gdp.drop_duplicates(subset=['Country'], keep='first')
else:
    print("The Population dataset was checked for duplicate entries based on the Country column.")
    print("Result: No duplicate country entries were found.")
    print("Therefore, no rows were removed or modified during this step.")


The Population dataset was checked for duplicate entries based on the Country column.
Result: No duplicate country entries were found.
Therefore, no rows were removed or modified during this step.


e)

In [62]:
# Make a copy of original names for mismatch logging
df_gdp["OriginalCountry"] = df_gdp["Country"]

# Normalize names
df_gdp["Country"] = (
    df_gdp["Country"]
    .str.strip()
    .str.replace(r"^the\s+", "", case=False, regex=True)  # remove "the "
    .str.title()
)

# Log any name mismatches due to normalization
gdp_name_mismatches = df_gdp[df_gdp["OriginalCountry"] != df_gdp["Country"]][
    ["OriginalCountry", "Country"]
]
gdp_name_mismatches.to_csv("../output/gdp_name_mismatches.csv", index=False)

f)

In [63]:
# Drop the temporary 'OriginalCountry' column
if "OriginalCountry" in df_gdp.columns:
    df_gdp.drop(columns="OriginalCountry", inplace=True)

# Set Country as the index for consistency
if df_gdp.index.name != "Country":
    df_gdp.set_index("Country", inplace=True)


## 4.3 Clean df_pop

a)

In [ ]:
# Ensure Population is numeric
df_pop = pd.read_csv('../population_2021.csv')

# Convert Population column to string to ensure compatibility with string operations
df_pop['Population'] = df_pop['Population'].astype(str)

# Remove all non-numeric characters using a regular expression
# This removes commas, spaces, and any other symbols except digits and decimal points
df_pop['Population'] = df_pop['Population'].str.replace(r"[^\d.]", "", regex=True)

# Convert the cleaned strings back to numeric values (floats)
# Invalid parsing will become NaN
df_pop['Population'] = pd.to_numeric(df_pop['Population'], errors='coerce')

# print(df_pop.head())

       Country  Population
0  Afghanistan    40000360
1       Africa  1413750475
2  Africa (UN)  1413753005
3      Albania     2849591
4      Algeria    44761051


b)

In [56]:
# Remove missing Population values

before_rows = len(df_pop)
df_pop = df_pop.dropna(subset=['Population'])
after_rows = len(df_pop)
rows_removed = before_rows - after_rows

# Log the number of rows removed
print(f"Number of rows removed due to missing population data: {rows_removed}")


Number of rows removed due to missing population data: 0


c)

In [ ]:
# Outlier detection on log-transformed population ===

# Log10-transformation to the population
df_pop['Log10_Population'] = np.log10(df_pop['Population'])

# Apply the Tukey method
Q1 = df_pop['Log10_Population'].quantile(0.25)
Q3 = df_pop['Log10_Population'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
pop_outliers = df_pop[
    (df_pop['Log10_Population'] < lower_bound) |
    (df_pop['Log10_Population'] > upper_bound)
]

print(f"Number of population outliers (after log10 transform): {len(pop_outliers)}")
# Print the outliers
print("Population outliers:\n", pop_outliers)

Number of population outliers (after log10 transform): 1
Population outliers:
      Country  Population  Log10_Population
251  Vatican         515          2.711807


d)

In [68]:
# Duplicate check and country name normalization ===

# Duplicate Handling Documentation
# Check for countries that appear more than once.
# If duplicates are found, we:
#   - Print the duplicate entries for review.
#   - Retain only the first occurrence of each country using `keep='first'`.
#     This is based on the assumption that the first entry is correct or representative,
#     and no additional criteria (e.g., recency, source reliability) are available to decide otherwise.
# This ensures consistency in merging/joining with other datasets later.

# Check for duplicates based on Country column
duplicates = df_pop[df_pop.duplicated(subset=['Country'], keep=False)]

if not duplicates.empty:
    print("Duplicate entries found:\n", duplicates)
    print("The Population dataset contains duplicate country entries.")
    print("Decision: Keeping only the first occurrence of each duplicated country.")
    df_pop = df_pop.drop_duplicates(subset=['Country'], keep='first')
else:
    print("The Population dataset was checked for duplicate entries based on the Country column.")
    print("Result: No duplicate country entries were found.")
    print("Therefore, no rows were removed or modified during this step.")

# Country name normalization (same logic as demographics & GDP datasets)
df_pop["OriginalCountry"] = df_pop["Country"]

df_pop["Country"] = (
    df_pop["Country"]
    .str.strip()
    .str.replace(r"^the\s+", "", case=False, regex=True)
    .str.title()
)

# Log mismatches between original and normalized names
name_mismatches = df_pop[df_pop["OriginalCountry"] != df_pop["Country"]][
    ["OriginalCountry", "Country"]
]
name_mismatches.to_csv("../output/pop_name_mismatches.csv", index=False)

The Population dataset was checked for duplicate entries based on the Country column.
Result: No duplicate country entries were found.
Therefore, no rows were removed or modified during this step.


e)

In [69]:
# Drop the temporary 'OriginalCountry' column
if "OriginalCountry" in df_gdp.columns:
    df_pop.drop(columns="OriginalCountry", inplace=True)

# Set Country as the index for consistency
if df_gdp.index.name != "Country":
    df_pop.set_index("Country", inplace=True)
